In [9]:
import pandas as pd
import os
import json


In [10]:
def read_folder(folder_path):
    """
    Opens all folder inside "folder_path" and reads the contents of thoose folders.
    """

    data_list = []

    for folder in os.listdir(folder_path):
        folderPath = folder_path+"/"+folder
        
        for file in os.listdir(folderPath):
            filePath = f"{folderPath}/{file}"
            with open( filePath ) as json_file:
                data = json.load(json_file)
                data["filePath"] = filePath

                data_list.append( data )
    
    return data_list


In [11]:
def dict_to_df(_dict:dict) -> pd.DataFrame:
    """Converts a dictionary to a Dataframe"""
    return pd.DataFrame( dict( zip(list( _dict ), [ [_dict[key]] for key in _dict]) ) )

def split_col_D(df, col,  prefix=None, splitter="_", drop_OG_col=True):
    """
    Splits a dataframe column only contain Dict, into a new one, where each key, in the dict is it's own column.

    Parameters
    ----------
    df: pd.DataFrame
        The DataFrame that contains "col"
    col: str
        The name of the columns, that is going to be split
    prefix: str or None, default None
        The prefix for naming the new columns, if None use "col" as prefix
    splitter: str, default "_"
        The string between prefix and the dict-key when making names for the new columns.
    drop_OG_col: bool. default True
        Wheter or not to drop the original columnm, when returning the df
    
    Returns
    -------
    pd.DataFrame now with a column split into mutiple columns

    Notes
    -----
    The result may contain NaN-values, in cases where not all the dict's contain the same keys.
    """

    if prefix == None: prefix = col    

    new_df = pd.concat( [dict_to_df(d) for d in df[col]] )

    rename_dict = dict(zip( [ col for col in new_df.columns] , [ prefix+splitter+col for col in new_df.columns] ))

    new_df = new_df.rename(columns=rename_dict)
    new_df.index = df.index

    target_col_I =  list(df.columns).index(col)
    new_cols = list(df.columns)[:target_col_I] + list(new_df.columns) + list(df.columns)[target_col_I+drop_OG_col:]
    new_cols

    return pd.concat([df, new_df], axis=1)[new_cols]

In [12]:
root = "../../"
cifar_df = pd.concat( [dict_to_df(i) for i in read_folder(root+r"Results\HPO\TripletAgain\Cifar")] )
cifar_df = split_col_D(cifar_df, "task")
cifar_df = split_col_D(cifar_df, "result")

best = []

for bit in cifar_df["task_bits"].unique():
    bit_df = cifar_df[cifar_df["task_bits"] == bit]
    
    best.append( bit_df[bit_df["result_map"] == max(bit_df["result_map"])] )

pd.concat(best)

,task_alpha,task_lr,task_wd,task_bits,result_earlyStop,result_loss,result_map,result_time stopage,time,filePath
0,3,0.0001,0.000010,12,True,"[512.3978039550781, 432.7353210449219, 409.824...",0.737452,NaN,1313.685490,../../Results\HPO\TripletAgain\Cifar/THOR-DESK...
0,5,0.0001,0.000100,48,True,"[900.6663262939453, 772.9544567871094, 704.761...",0.786867,NaN,1553.986386,../../Results\HPO\TripletAgain\Cifar/THOR-DESK...
0,3,0.0001,0.000001,32,True,"[508.16312438964843, 421.3442919921875, 384.36...",0.802379,NaN,1156.970827,../../Results\HPO\TripletAgain\Cifar/THOR-DESK...
0,3,0.0001,0.000100,24,True,"[508.29256927490235, 427.44655151367186, 398.8...",0.783474,NaN,1527.279399,../../Results\HPO\TripletAgain\Cifar/THOR-DESK...


In [13]:
cifar_df = pd.concat( [dict_to_df(i) for i in read_folder(root+r"Results\HPO\TripletAgain\ImgNet")] )
cifar_df = split_col_D(cifar_df, "task")
cifar_df = split_col_D(cifar_df, "result")

best = []

for bit in cifar_df["task_bits"].unique():
    bit_df = cifar_df[cifar_df["task_bits"] == bit]
    
    best.append( bit_df[bit_df["result_map"] == max(bit_df["result_map"])] )

pd.concat(best)

,task_alpha,task_lr,task_wd,task_bits,result_earlyStop,result_loss,result_map,result_time stopage,time,filePath
0,5,0.00010,0.00001,32,True,"[8.31062403535843, 7.086685581207275, 6.251981...",0.058203,NaN,3255.658049,../../Results\HPO\TripletAgain\ImgNet/THOR-DES...
0,1,0.00001,0.00010,48,True,"[1.8196791911125183, 1.5443865611553191, 1.495...",0.073928,NaN,3162.523780,../../Results\HPO\TripletAgain\ImgNet/THOR-DES...
0,5,0.00010,0.00001,12,True,"[8.222869109869004, 6.804663819313049, 6.33404...",0.024952,NaN,2783.977311,../../Results\HPO\TripletAgain\ImgNet/THOR-DES...
0,5,0.00010,0.00001,24,NaN,"[8.802125539779663, 6.645236833572388, 6.66264...",0.047201,True,3386.073003,../../Results\HPO\TripletAgain\ImgNet/THOR-DES...


In [14]:
cifar_df = pd.concat( [dict_to_df(i) for i in read_folder(root+r"Results\HPO\TripletAgain\Nus")] )
cifar_df = split_col_D(cifar_df, "task")
cifar_df = split_col_D(cifar_df, "result")

best = []

for bit in cifar_df["task_bits"].unique():
    bit_df = cifar_df[cifar_df["task_bits"] == bit]
    
    best.append( bit_df[bit_df["result_map"] == max(bit_df["result_map"])] )

pd.concat(best)

,task_alpha,task_lr,task_wd,task_bits,result_time stopage,result_loss,result_map,time,filePath
0,5,0.0001,0.00001,32,True,"[2328.1172841893563, 1850.2960918162128, 1698....",0.771851,1804.164381,../../Results\HPO\TripletAgain\Nus/THOR-DESKTO...
0,3,0.0001,0.00001,12,True,"[1214.492660673538, 995.6088117844988, 946.483...",0.742190,1815.522821,../../Results\HPO\TripletAgain\Nus/THOR-DESKTO...
0,3,0.0001,0.00010,48,True,"[1162.0352493135056, 941.6801401270499, 899.33...",0.786261,1807.863233,../../Results\HPO\TripletAgain\Nus/THOR-DESKTO...
0,5,0.0001,0.00001,24,True,"[2293.710915140586, 1847.2270822052908, 1725.3...",0.767628,1808.388321,../../Results\HPO\TripletAgain\Nus/THOR-DESKTO...


In [16]:
cifar_df = pd.concat( [dict_to_df(i) for i in read_folder(root+r"Results\HPO\DTSH2\Nus")] )
cifar_df = split_col_D(cifar_df, "task")
cifar_df = split_col_D(cifar_df, "result")

best = []

for bit in cifar_df["task_bits"].unique():
    bit_df = cifar_df[cifar_df["task_bits"] == bit]
    
    best.append( bit_df[bit_df["result_map"] == max(bit_df["result_map"])] )

pd.concat(best)

,task_lambda,task_alpha,task_lr,task_wd,task_bits,result_earlyStop,result_loss,result_map,time,filePath
0,0.5,3,0.00010,0.00001,24,True,"[2.2154292116562524, 1.1645363548807068, 1.075...",0.805781,445.319330,../../Results\HPO\DTSH2\Nus/THOR-DESKTOP/2024-...
0,0.5,3,0.00010,0.00001,48,True,"[2.377163570885565, 1.2449635845773361, 1.1329...",0.814146,466.806876,../../Results\HPO\DTSH2\Nus/THOR-DESKTOP/2024-...
0,0.5,3,0.00010,0.00001,32,True,"[2.1949620819559286, 1.233482977923225, 1.1397...",0.809201,472.482828,../../Results\HPO\DTSH2\Nus/THOR-DESKTOP/2024-...
0,0.5,5,0.00001,0.00001,12,True,"[2.574555894907783, 2.010723203566729, 1.89887...",0.787345,735.136782,../../Results\HPO\DTSH2\Nus/THOR-DESKTOP/2024-...


In [17]:
cifar_df = pd.concat( [dict_to_df(i) for i in read_folder(root+r"Results\HPO\DTSH2\ImgNet")] )
cifar_df = split_col_D(cifar_df, "task")
cifar_df = split_col_D(cifar_df, "result")

best = []

for bit in cifar_df["task_bits"].unique():
    bit_df = cifar_df[cifar_df["task_bits"] == bit]
    
    best.append( bit_df[bit_df["result_map"] == max(bit_df["result_map"])] )

pd.concat(best)

,task_lambda,task_alpha,task_lr,task_wd,task_bits,result_time stopage,result_loss,result_map,result_earlyStop,time,filePath
0,0.5,5,0.00010,0.000001,24,True,"[0.4950497688949108, 0.27191812255978587, 0.24...",0.078484,NaN,3459.798236,../../Results\HPO\DTSH2\ImgNet/THOR-DESKTOP/20...
0,0.5,5,0.00010,0.000010,32,True,"[0.42961945205926894, 0.2788698516190052, 0.23...",0.087018,NaN,3503.843179,../../Results\HPO\DTSH2\ImgNet/THOR-DESKTOP/20...
0,0.5,5,0.00001,0.000010,48,True,"[0.3036577241420746, 0.24711909338831903, 0.23...",0.089897,NaN,3752.676340,../../Results\HPO\DTSH2\ImgNet/THOR-DESKTOP/20...
0,0.5,3,0.00001,0.000001,12,True,"[0.403533907353878, 0.31496368724107743, 0.290...",0.038556,NaN,3297.245387,../../Results\HPO\DTSH2\ImgNet/THOR-DESKTOP/20...


In [19]:
cifar_df = pd.concat( [dict_to_df(i) for i in read_folder(root+r"Results\HPO\DTSH2\Cifar")] )
cifar_df = split_col_D(cifar_df, "task")
cifar_df = split_col_D(cifar_df, "result")

best = []

for bit in cifar_df["task_bits"].unique():
    bit_df = cifar_df[cifar_df["task_bits"] == bit]
    
    best.append( bit_df[bit_df["result_map"] == max(bit_df["result_map"])] )

pd.concat(best)

,task_lambda,task_alpha,task_lr,task_wd,task_bits,result_earlyStop,result_loss,result_map,result_time stopage,time,filePath
0,2.0,5,0.00001,0.00001,48,NaN,"[2.6224216771125795, 2.043395359516144, 1.9079...",0.869544,True,1802.365761,../../Results\HPO\DTSH2\Cifar/THOR-DESKTOP/202...
0,0.5,5,0.00010,0.00010,12,True,"[2.3410697770118714, 1.4381213390827179, 1.315...",0.847115,NaN,268.011624,../../Results\HPO\DTSH2\Cifar/THOR-DESKTOP/202...
0,1.0,3,0.00001,0.00010,24,NaN,"[1.5654631018638612, 1.2056463623046876, 1.123...",0.862090,True,1801.745347,../../Results\HPO\DTSH2\Cifar/THOR-DESKTOP/202...
0,0.5,5,0.00001,0.00010,32,NaN,"[2.146756012439728, 1.6190512418746947, 1.4825...",0.872454,True,1801.770384,../../Results\HPO\DTSH2\Cifar/THOR-DESKTOP/202...
